# Tutorial 1 - Linear Regression

We will predict the price (`price` column) of an AirBNB listing in Boston given a number of features about the listing.

**Therefore, our unit of analysis is an AIRBNB LISTING**

# Setup

In [61]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(142)


# Get the data

In [62]:
#We will predict the "price" value in the data set:

airbnb = pd.read_csv("/Users/harshrajchauhan/Downloads/DSP/Module 2/Material/airbnb.csv")
airbnb.head()

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_$75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_$75
3,0,0,Roslindale,42.281106,-71.121021,House,Private room,4,1.0,1.0,...,2,25,1,1,0,100.0,moderate,75,0,lte_$75
4,1,1,Roslindale,42.284512,-71.136258,House,Private room,2,1.5,1.0,...,1,0,2,29,380,99.0,flexible,79,0,btw_$75-$150


In [63]:
airbnb.shape

(3555, 23)

# Split the data into train and test

In [64]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(airbnb, test_size=0.3)

### Be careful: we haven't seperated the target column yet

## Check the missing values

In [65]:
train_set.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          3
room_type                              0
accommodates                           0
bathrooms                              8
bedrooms                               8
beds                                   5
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 569
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

In [66]:
test_set.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                              6
bedrooms                               2
beds                                   4
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 231
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

# Data Prep

In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Drop the variables we can't use in this tutorial

In [68]:
# We can't use the following columns in this tutorial, because they are for classification tasks

train = train_set.drop(['price_gte_150', 'price_category'], axis=1)
test = test_set.drop(['price_gte_150', 'price_category'], axis=1)

## Separate the target variable (we don't want to transform it)

In [69]:
train_y = train[['price']]
test_y = test[['price']]

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

##  Identify the numerical and categorical columns

### Option 1: Manually

### Option 2: Programmatically

In [70]:
train_inputs.dtypes

host_is_superhost                      int64
host_identity_verified                 int64
neighbourhood_cleansed                object
latitude                             float64
longitude                            float64
property_type                         object
room_type                             object
accommodates                           int64
bathrooms                            float64
bedrooms                             float64
beds                                 float64
bed_type                              object
Number of amenities                    int64
guests_included                        int64
price_per_extra_person                 int64
minimum_nights                         int64
number_of_reviews                      int64
number_days_btw_first_last_review      int64
review_scores_rating                 float64
cancellation_policy                   object
dtype: object

In [71]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [72]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['host_is_superhost', 'host_identity_verified']

In [73]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [74]:
binary_columns

['host_is_superhost', 'host_identity_verified']

In [75]:
numeric_columns

['latitude',
 'longitude',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'Number of amenities',
 'guests_included',
 'price_per_extra_person',
 'minimum_nights',
 'number_of_reviews',
 'number_days_btw_first_last_review',
 'review_scores_rating']

In [76]:
categorical_columns

['neighbourhood_cleansed',
 'property_type',
 'room_type',
 'bed_type',
 'cancellation_policy']

# Pipeline

In [77]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [78]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [79]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [80]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='drop')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [81]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.57516407, -0.16927383, -1.15578785, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.1958557 ,  0.18733179, -0.58455222, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.42207037,  0.69457073, -0.01331659, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.04321532, -0.1499537 , -1.15578785, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.7776294 , -1.47612124, -0.58455222, ...,  0.        ,
         1.        ,  0.        ],
       [-1.41278267, -0.78143866, -0.58455222, ...,  0.        ,
         0.        ,  1.        ]])

In [82]:
train_x.shape

(2488, 66)

# Tranform: transform() for TEST

In [83]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 1.39510971,  1.60175586,  1.70039032, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.77451956, -1.74858009,  1.70039032, ...,  0.        ,
         0.        ,  1.        ],
       [-0.05462059,  0.13283793,  0.55791905, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.60448836,  0.73614916,  1.12915468, ...,  0.        ,
         0.        ,  0.        ],
       [-0.99210521, -0.23519825, -1.15578785, ...,  0.        ,
         1.        ,  1.        ],
       [-1.44872042, -0.7116745 , -0.58455222, ...,  0.        ,
         0.        ,  1.        ]])

In [84]:
test_x.shape

(1067, 66)

# Calculate the baseline

In [25]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_y)

DummyRegressor()

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
# This is the baseline Train RMSE

dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_y, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 102.49847834318642


In [28]:
# This is the baseline Test RMSE

dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_y, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 105.36001777194679


# Linear Regression Using Stochastic Gradient Descent

In [29]:
from sklearn.linear_model import SGDRegressor 

# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.01) 

sgd_reg.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(max_iter=100, penalty=None)

# Predicted vs. Actual values

In [30]:
sgd_reg.predict(test_x)

array([238.20810252, 237.27412787, 202.73894752, ..., 169.48965419,
        96.98753928,  73.81986261])

In [31]:
# Create a new DataFrame

predictions = pd.DataFrame(sgd_reg.predict(test_x), columns=['Predicted'])

predictions

,Predicted
0,238.208103
1,237.274128
2,202.738948
3,213.174048
4,77.685773
...,...
1062,161.522131
1063,146.283552
1064,169.489654
1065,96.987539


In [32]:
# Add the actual to the same DataFrame

predictions['Actual'] = np.array(test_y)

predictions

,Predicted,Actual
0,238.208103,300
1,237.274128,283
2,202.738948,148
3,213.174048,289
4,77.685773,74
...,...,...
1062,161.522131,170
1063,146.283552,137
1064,169.489654,125
1065,96.987539,35


# Calculate the overall error

In [33]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 62.934856346724104


In [34]:
#Test RMSE
reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 65.05032283486635


# L2 Regularization

In [35]:
#Stochastic Gradient:

sgd_reg_L2 = SGDRegressor(max_iter=50, penalty='l2', alpha = 0.1, eta0=0.01)

sgd_reg_L2.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, max_iter=50)

In [36]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 66.33825281909766


In [37]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 69.30459941994073


# L1 Regularization

In [38]:
#Stochastic Gradient:
sgd_reg_L1 = SGDRegressor(max_iter=50, penalty='l1', alpha = 0.1, eta0=0.01)

sgd_reg_L1.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=0.1, max_iter=50, penalty='l1')

In [39]:
#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 62.9969790415513


In [40]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 65.31553917706802


# ElasticNet

In [41]:
#Stochastic Gradient:
sgd_reg_elastic = SGDRegressor(max_iter=50, penalty='elasticnet', l1_ratio=0.5, alpha = 0.1, 
                          eta0=0.01)
sgd_reg_elastic.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, l1_ratio=0.5, max_iter=50, penalty='elasticnet')

In [42]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 65.07952121269928


In [43]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 67.69041494068347


# Early Stopping

In [44]:
np.random.seed(42)

# tol is the early stopping criteria

sgd_reg_ES = SGDRegressor(max_iter=500, early_stopping = True, n_iter_no_change=5, tol=0.0001, 
                          validation_fraction=0.2, eta0=0.01)

In [45]:
sgd_reg_ES.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(early_stopping=True, max_iter=500, tol=0.0001,
             validation_fraction=0.2)

### Number of iterations with early stopping

In [46]:
sgd_reg_ES.n_iter_

36

In [47]:
#Train RMSE
reg_train_pred = sgd_reg_ES.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 63.10285931655026


In [48]:
#Test RMSE
reg_test_pred = sgd_reg_ES.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 65.35306833610807


### Number of iterations without early stopping

In [49]:
np.random.seed(42)

sgd_reg_ES = SGDRegressor(max_iter=500, eta0=0.01)

In [50]:
sgd_reg_ES.fit(train_x, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(max_iter=500)

In [51]:
sgd_reg_ES.n_iter_

54

In [52]:
#Train RMSE
reg_train_pred = sgd_reg_ES.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 62.78974394639683


In [53]:
#Test RMSE
reg_test_pred = sgd_reg_ES.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 65.20502066134019


# Polynomial Regression

This is done by creating the polynomial "variables" of the existing variables, then fitting them in a regular regression model


In [54]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)

#This will create the polynomial terms of the categorical variables too

#if degree=3, then it creates all combinations: a, a^2, a^3, b, b^2, b^3, a.b, a^2.b, a.b^2, a^2.b^2 

In [55]:
#We still fit a linear regression model

pol_lin_reg = SGDRegressor(max_iter=1000, penalty=None, eta0=0.01) 

pol_lin_reg.fit(train_x_poly, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(penalty=None)

In [56]:
#Train RMSE
reg_train_pred = pol_lin_reg.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 10430157565756.498


In [57]:
#Test RMSE
reg_test_pred = pol_lin_reg.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 910218017002.7522


## Let's Peform Regularization using ElasticNet

In [58]:
# Remember, alpha is the magnitude of regularization
# And, l1_ratio is how much L1 vs. L2 regularization to do.

# INCREASE THE ALPHA VALUE TO CORRECT OVERFITTING

elastic_net = SGDRegressor(max_iter=1000, penalty='elasticnet', l1_ratio=0.5, alpha = 1, 
                          eta0=0.01)

elastic_net.fit(train_x_poly, train_y)

/Users/harshrajchauhan/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=1, l1_ratio=0.5, penalty='elasticnet')

In [59]:
#Train RMSE
reg_train_pred = elastic_net.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 7750745352911.481


In [60]:
#Test RMSE
reg_test_pred = elastic_net.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 566101641581.0092
